# 02 - Black-Scholes and Implied Volatility

Validate parity, compute Greeks, and recover implied volatility from synthetic prices.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd

from src.config import config_dict
from src.black_scholes import bs_call_price, bs_put_price, bs_delta, bs_gamma, bs_vega
from src.implied_vol import implied_vol
from src.plotting import plot_implied_vol_recovery

cfg = config_dict(fast_mode=True)


In [ ]:
strikes = np.linspace(80, 120, 9)
parity_rows = []
for K in strikes:
    C = bs_call_price(cfg['S0'], float(K), cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'])
    P = bs_put_price(cfg['S0'], float(K), cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'])
    rhs = cfg['S0'] * np.exp(-cfg['Q'] * cfg['T']) - float(K) * np.exp(-cfg['R'] * cfg['T'])
    parity_rows.append({'strike': K, 'parity_residual': C - P - rhs})
parity_df = pd.DataFrame(parity_rows)
parity_df.head()


In [ ]:
sigma_true = cfg['SIGMA']
rows = []
for K in strikes:
    price = bs_call_price(cfg['S0'], float(K), cfg['R'], cfg['Q'], sigma_true, cfg['T'])
    iv = implied_vol(
        price=price,
        S=cfg['S0'],
        K=float(K),
        r=cfg['R'],
        q=cfg['Q'],
        T=cfg['T'],
        option_type='call',
    )
    rows.append({'strike': float(K), 'price': price, 'implied_vol': iv, 'sigma_true': sigma_true})

iv_df = pd.DataFrame(rows)
iv_df.to_csv(ROOT / 'results' / 'tables' / 'implied_vol_recovery.csv', index=False)
plot_implied_vol_recovery(iv_df, ROOT / 'results' / 'figures' / 'implied_vol_recovery.png')

greeks = {
    'delta_call': bs_delta(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'], 'call'),
    'gamma': bs_gamma(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T']),
    'vega': bs_vega(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T']),
}
print(greeks)
iv_df
